In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_7/models/B5R2b5_5FP_1FC_fold7_1.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold7.csv') #Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(4600, 25)
(4600, 25)
Normal:  (3005, 25)
Abnormal:  (1595, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG,AB01 P3-1 C015.JPG
1,1,1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG,AB01 P1 C015.JPG
2,2,2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG,AB01 P2 C015.JPG
3,3,3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG,AB01 P4-2 C015.JPG
4,4,4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Views',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 4600 validated image filenames belonging to 5 classes.
{0: 'FP-A', 1: 'FP-B', 2: 'FP-', 3: 'FP-D', 4: 'FP-E'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[ 9.86294627e-01,  2.14492582e-04, -1.88388482e-01, ...,
        -1.86914112e-02,  2.07206503e-01,  9.18636024e-01],
       [ 2.36491024e-01,  8.84336770e-01,  1.42358697e-03, ...,
        -1.28793254e-01, -1.62727624e-01,  7.90093660e-01],
       [-2.47435682e-02,  6.09713554e-01, -5.78028522e-02, ...,
        -5.51765189e-02, -1.81716159e-01,  9.14777696e-01],
       ...,
       [-1.83154568e-01, -1.77660808e-01,  4.42956030e-01, ...,
         1.74360320e-01,  2.61467129e-01, -1.23359874e-01],
       [-2.03872085e-01, -2.00695574e-01,  3.39430958e-01, ...,
         4.55701381e-01,  6.09110534e-01, -5.92436902e-02],
       [-1.23817496e-01, -2.23008871e-01, -2.19725311e-01, ...,
        -7.90501311e-02, -2.24806160e-01,  8.47748816e-02]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.986295,0.000214,-0.188388,-0.088482,-0.010367,0.424080,-0.112542,0.264733,-0.136909,0.976137,...,-0.029746,-0.081793,0.177616,0.771813,0.011231,0.662107,0.186359,-0.018691,0.207207,0.918636
1,0.236491,0.884337,0.001424,0.116625,0.468836,0.041453,1.049583,0.551325,-0.031517,0.234770,...,0.119858,-0.115412,-0.189681,0.288450,-0.109788,0.270928,0.725451,-0.128793,-0.162728,0.790094
2,-0.024744,0.609714,-0.057803,0.567234,0.702744,-0.176519,1.007505,0.066433,-0.114292,-0.097552,...,0.261161,-0.167651,-0.188022,0.034733,-0.178272,0.166651,0.780952,-0.055177,-0.181716,0.914778
3,0.163779,-0.196792,0.010655,0.037266,0.005610,0.441722,-0.053995,0.286697,-0.103555,0.605432,...,0.147132,0.006986,-0.044373,0.638993,-0.180175,0.070436,-0.199377,0.087352,0.147240,0.552130
4,0.804756,0.100611,-0.142152,-0.020464,0.085910,1.049034,-0.141936,0.542180,-0.020645,0.918757,...,0.307526,-0.132033,0.181096,0.787678,-0.180178,0.586436,-0.042483,0.119813,0.101700,0.963566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,0.580081,0.081157,-0.202055,-0.055976,-0.038843,0.492340,-0.221081,0.202125,-0.211172,0.944930,...,0.037834,-0.211970,0.020482,0.539458,-0.201034,0.856926,0.299519,-0.062546,-0.211236,0.801512
4596,0.163949,-0.138466,0.005822,-0.029014,0.069206,0.480572,-0.203401,0.243316,-0.040707,0.701990,...,0.100306,0.112921,0.417831,0.580988,-0.111179,0.085379,-0.026939,0.201886,0.154778,0.837904
4597,-0.183155,-0.177661,0.442956,0.408842,-0.182873,-0.219403,-0.187975,-0.091318,0.320332,-0.222782,...,0.264739,0.122473,0.187148,-0.057464,-0.007473,-0.174601,-0.214004,0.174360,0.261467,-0.123360
4598,-0.203872,-0.200696,0.339431,0.471467,0.163014,-0.194705,-0.198339,0.147333,0.560823,-0.132336,...,0.042063,0.106959,0.490950,0.180368,0.754900,-0.126592,-0.221690,0.455701,0.609111,-0.059244


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,0.986295,0.000214,-0.188388,-0.088482,-0.010367,0.424080,-0.112542,0.264733,-0.136909,0.976137,...,0.011231,0.662107,0.186359,-0.018691,0.207207,0.918636,Abnormal,P31,FP-B,AB01
1,0.236491,0.884337,0.001424,0.116625,0.468836,0.041453,1.049583,0.551325,-0.031517,0.234770,...,-0.109788,0.270928,0.725451,-0.128793,-0.162728,0.790094,Abnormal,P1,FP-A,AB01
2,-0.024744,0.609714,-0.057803,0.567234,0.702744,-0.176519,1.007505,0.066433,-0.114292,-0.097552,...,-0.178272,0.166651,0.780952,-0.055177,-0.181716,0.914778,Abnormal,P2,FP-A,AB01
3,0.163779,-0.196792,0.010655,0.037266,0.005610,0.441722,-0.053995,0.286697,-0.103555,0.605432,...,-0.180175,0.070436,-0.199377,0.087352,0.147240,0.552130,Abnormal,P42,FP-B,AB01
4,0.804756,0.100611,-0.142152,-0.020464,0.085910,1.049034,-0.141936,0.542180,-0.020645,0.918757,...,-0.180178,0.586436,-0.042483,0.119813,0.101700,0.963566,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,0.580081,0.081157,-0.202055,-0.055976,-0.038843,0.492340,-0.221081,0.202125,-0.211172,0.944930,...,-0.201034,0.856926,0.299519,-0.062546,-0.211236,0.801512,Normal,P32,FP-B,Normal
4596,0.163949,-0.138466,0.005822,-0.029014,0.069206,0.480572,-0.203401,0.243316,-0.040707,0.701990,...,-0.111179,0.085379,-0.026939,0.201886,0.154778,0.837904,Normal,P42,FP-B,Normal
4597,-0.183155,-0.177661,0.442956,0.408842,-0.182873,-0.219403,-0.187975,-0.091318,0.320332,-0.222782,...,-0.007473,-0.174601,-0.214004,0.174360,0.261467,-0.123360,Normal,P52,FP-C,Normal
4598,-0.203872,-0.200696,0.339431,0.471467,0.163014,-0.194705,-0.198339,0.147333,0.560823,-0.132336,...,0.754900,-0.126592,-0.221690,0.455701,0.609111,-0.059244,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_5FP_fold7_1.csv') #เปลี่ยนชื่อไฟล์